In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('Audiobooks_data.csv',delimiter=',')

In [3]:
df.head()

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,4.61,0,8.91,0.00,0.0,0,0,0


In [4]:
df['0'].value_counts()

0    11846
1     2237
Name: 0, dtype: int64

In [5]:
from sklearn.utils import resample

In [6]:
# Separate majority and minority classes
df_majority = df[df['0']==0]
df_minority = df[df['0']==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=11846,    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled['0'].value_counts()

1    11846
0    11846
Name: 0, dtype: int64

In [7]:
df_upsampled.shape

(23692, 12)

In [8]:
df_upsampled

,00994,1620,1620.1,19.73,19.73.1,1,10.00,0.99,1603.80,5,92,0
0,1143,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,0,0
1,2059,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,388,0
2,2882,1620.0,1620,5.96,5.96,0,8.91,0.42,680.4,1,129,0
3,3342,2160.0,2160,5.33,5.33,0,8.91,0.22,475.2,0,361,0
4,3416,2160.0,2160,4.61,4.61,0,8.91,0.00,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
13051,31052,1890.0,3780,6.13,12.26,0,8.91,0.00,0.0,0,24,1
13712,31427,2160.0,2160,5.68,5.68,0,8.91,0.00,0.0,0,164,1
130,11582,2160.0,2160,5.33,5.33,0,8.91,0.00,0.0,0,152,1
3834,12679,1620.0,1620,7.83,7.83,0,8.91,0.00,0.0,0,66,1


In [9]:
X = df_upsampled.iloc[:,1:-1].values

In [10]:
y = df_upsampled.iloc[:,-1].values

In [11]:
y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [14]:
 X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [15]:
from sklearn.preprocessing import StandardScaler

In [16]:
scaler =StandardScaler()

In [17]:
X_train = scaler.fit_transform(X_train)

In [18]:
X_test = scaler.transform(X_test)

In [19]:
X_val = scaler.transform(X_val)

In [20]:
import tensorflow

C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Raaz\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [22]:
model = Sequential()

In [23]:
model.add(Dense(units=50,activation='relu'))
model.add(Dense(units=50,activation='relu'))
model.add(Dense(units=2,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [24]:
early_stopping = tensorflow.keras.callbacks.EarlyStopping(patience=4)

In [25]:
model.fit(X_train,y_train,epochs=100,batch_size=100,callbacks=[early_stopping],validation_data=(X_val,y_val),verbose=2)

Train on 14214 samples, validate on 4739 samples
Epoch 1/100
14214/14214 - 1s - loss: 0.5245 - acc: 0.7227 - val_loss: 0.4171 - val_acc: 0.7858
Epoch 2/100
14214/14214 - 0s - loss: 0.3867 - acc: 0.8010 - val_loss: 0.3800 - val_acc: 0.8012
Epoch 3/100
14214/14214 - 1s - loss: 0.3674 - acc: 0.8084 - val_loss: 0.3728 - val_acc: 0.8027
Epoch 4/100
14214/14214 - 0s - loss: 0.3607 - acc: 0.8110 - val_loss: 0.3634 - val_acc: 0.8054
Epoch 5/100
14214/14214 - 1s - loss: 0.3564 - acc: 0.8116 - val_loss: 0.3800 - val_acc: 0.7824
Epoch 6/100
14214/14214 - 1s - loss: 0.3515 - acc: 0.8146 - val_loss: 0.3590 - val_acc: 0.8124
Epoch 7/100
14214/14214 - 1s - loss: 0.3500 - acc: 0.8160 - val_loss: 0.3545 - val_acc: 0.8111
Epoch 8/100
14214/14214 - 1s - loss: 0.3467 - acc: 0.8163 - val_loss: 0.3555 - val_acc: 0.8073
Epoch 9/100
14214/14214 - 1s - loss: 0.3458 - acc: 0.8169 - val_loss: 0.3515 - val_acc: 0.8114
Epoch 10/100
14214/14214 - 0s - loss: 0.3448 - acc: 0.8198 - val_loss: 0.3582 - val_acc: 0.8088


In [26]:
pred = model.predict_classes(X_test)

In [27]:
pred

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [28]:
test_loss,test_accuracy = model.evaluate(X_test,y_test)

4739/4739 [==============================] - 0s 76us/sample - loss: 0.3401 - acc: 0.8225


In [29]:
print('test_loss:',test_loss,'test_accuracy:',test_accuracy)

test_loss: 0.3400894368052055 test_accuracy: 0.8225364
